# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [82]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [83]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Date,Country
0,0,Port Alfred,-33.59,26.89,18.23,28,4.87,0,2020-07-12,ZA
1,1,Busselton,-33.65,115.33,16.11,45,10.55,1,2020-07-12,AU
2,2,Puerto Ayora,-0.74,-90.35,18.33,97,1.51,61,2020-07-12,EC
3,3,New Norfolk,-42.78,147.06,5.56,89,0.10,80,2020-07-12,AU
4,4,Jamestown,42.10,-79.24,18.33,100,2.60,1,2020-07-12,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [84]:
gmaps.configure(api_key=g_key)

In [85]:
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)
fig = gmaps.figure(center=(35,135),zoom_level=3)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [88]:
idealcity_df = cities_df[(cities_df["Temperature"] >20)&(cities_df["Temperature"] <30)&(cities_df["Humidity"] <30)&(cities_df["Cloudiness"] <40)&(cities_df["Wind Speed"] <6)]

idealcity_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Date,Country
228,228,Saint George,37.10,-113.58,29.00,21,1.38,1,2020-07-12,US
281,281,Chipata,-13.63,32.65,28.46,25,4.59,0,2020-07-12,ZM
301,301,Price,39.60,-110.81,21.00,28,3.10,1,2020-07-12,US
325,325,Chingola,-12.53,27.88,28.52,24,3.45,0,2020-07-12,ZM
366,366,Chiredzi,-21.05,31.67,22.08,26,4.12,0,2020-07-12,ZW
430,430,Ondjiva,-17.07,15.73,22.00,24,4.10,0,2020-07-12,AO
473,473,Hurricane,37.18,-113.29,28.00,24,1.50,1,2020-07-12,US
480,480,Mwinilunga,-11.74,24.43,29.39,24,3.30,0,2020-07-12,ZM


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
hotel_df = idealcity_df[["City","Latitude","Longitude","Country"]]

In [90]:
hotel_name = []
hotel_rating = []
hotel_address = []

for index, row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    parameter = {"location":f'{row["Latitude"]},{row["Longitude"]}',"keyword":"hotel","radius":5000,"key":g_key}
    response = requests.get(url, params = parameter)
    data = response.json()
   
    
    try:
        hotel_name.append(data["results"][0]["name"])
        hotel_address.append(data["results"][0]['vicinity'])
        hotel_rating.append(data["results"][0]['rating'])
        print(f'Found a hotel in {row["City"]}.')
    except:
        hotel_name.append("Not found")
        hotel_address.append("Not found")
        hotel_rating.append("Not found")
        print(f'{row["City"]} has no hotel near the city.') 

hotel_df["Hotel Name"] = hotel_name
hotel_df["Hotel Rating"] = hotel_rating
hotel_df["Hotel Address"] = hotel_address



Found a hotel in Saint George.
Found a hotel in Chipata.
Found a hotel in Price.
Found a hotel in Chingola.
Found a hotel in Chiredzi.
Found a hotel in Ondjiva.
Found a hotel in Hurricane.
Found a hotel in Mwinilunga.


C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [92]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != "Not found"]
hotel_df

,City,Latitude,Longitude,Country,Hotel Name,Hotel Rating,Hotel Address
228,Saint George,37.10,-113.58,US,Inn on The Cliff St. George,4.8,"511 S Airport Rd, St. George"
281,Chipata,-13.63,32.65,ZM,Protea Hotel by Marriott Chipata,4.3,3126 Great East Road Chipata
301,Price,39.60,-110.81,US,Ramada by Wyndham Price,3.6,"838 Westwood Blvd, Price"
325,Chingola,-12.53,27.88,ZM,Protea Hotel by Marriott Chingola,4.1,"Kabundi Rd, Chingola"
366,Chiredzi,-21.05,31.67,ZW,The Nesbitt Arms,3.5,"238 Marula Dr, Chiredzi"
430,Ondjiva,-17.07,15.73,AO,Águia Verde Hotel,3.3,Ondjiva
473,Hurricane,37.18,-113.29,US,Best Western Plus Zion West Hotel,4.4,"44 W 500 N, La Verkin"
480,Mwinilunga,-11.74,24.43,ZM,Muzina Guest House,3.2,Mwinilunga


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """

<dl style="color:black;">
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]].astype(float)

In [94]:
# Add marker layer ontop of heat map

marker_layer = gmaps.symbol_layer(
    locations, fill_color='green', stroke_color='green', scale=5,
    info_box_content=hotel_info
)
# Display figure
fig = gmaps.figure(center=(40,-30),zoom_level=2)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))